In [1]:
# So as Dr. Pacagnell's comments indicate, the solution he implemented (that I adapted as solution2
# in the preceding exercise) does actually implement the full attack.  
# This code demonstrates it working with 1024 bit key.

from RSA import RSAClient, RSAServer, BIT_STRENGTH
from Crypto import Random

server = RSAServer()
client = RSAClient(**server.GetPubkey())

In [2]:
# First make sure all the RSA code is working

message = b'A'*(BIT_STRENGTH//8)
ciphertext = client.Encrypt(message)
result = server.DecryptBytes(ciphertext)

print(result)
print(len(result))
assert result[-len(message):] == message

b'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
128


In [3]:
Ceil  = lambda numerator, denominator: (numerator + denominator - 1) // denominator
Floor = lambda numerator, denominator: Ceil(numerator, denominator) - 1

assert  Ceil(10, 3) == 4
assert Floor(10, 3) == 3

In [4]:
class Oracle(RSAServer):
    def IsPaddingCorrect(self, encrypted_int_data):
        plaintext = self.DecryptBytes(encrypted_int_data)
        return len(plaintext) == Ceil(self.n.bit_length(), 8) and plaintext[:2] == b'\x00\x02'
    
def Pad(binary_data, length = BIT_STRENGTH//8):
    padding_string = Random.new().read(length - 3 - len(binary_data))
    return b'\x00\x02' + padding_string + b'\x00' + binary_data

Pad(b'Hello, world!')

b"\x00\x02\x86\xd4?\xc3ew\xef\xbb\x13\x1e\x07\xeft\xa5\x1f\xf8\xace\x18;TDF\xb87\xa5\x1b\x99\xfc\xb5\x10 @\xceCL\xddd\xf8\xdac\xff\x1f\x9c\xef\xeb\xb7.\xde\xa0X\xed\xc8\xa82\xa6z\xacyp\xa1\x84\x1f\xdf\xed&A\xea\x84\x1e'^\x9d\xfe\xde\xf1\x85\xa7\xe2B\xa7\x88l\xfa\xc6\x1c9\xa4\xaf\x9b\xfb\x10\x9c\xc6\xe2\xee\x976?\xd8Zp\xd1<\xa5yM\x82\xf5<vu\x00Hello, world!"

In [5]:
# Set up the attack
oracle = Oracle()
pubkey = oracle.GetPubkey()
e, n = pubkey['e'], pubkey['n']
client = RSAClient(**pubkey)
message = b'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.'
ciphertext = client.Encrypt(Pad(message))

B = 2**(BIT_STRENGTH-2*8)

In [6]:
# Do the attack

c_0 = ciphertext
M = [(2 * B, 3 * B - 1)]
i = 1

if not oracle.IsPaddingCorrect(c_0):
    # This shouldn't happen
    assert False
    while True:
        s = randint(0, n - 1)
        c_0 = (ciphertext * pow(s, e, n)) % n
        if oracle.IsPaddingCorrect(c_0):
            break

In [7]:
def CalculateNewM(intervals, lower_bound, upper_bound):

    for i, (a, b) in enumerate(intervals):

        # If there is an overlap, then replace the boundaries of the overlapping
        # interval with the wider (or equal) boundaries of the new merged interval
        if not (b < lower_bound or a > upper_bound):
            new_a = min(lower_bound, a)
            new_b = max(upper_bound, b)
            intervals[i] = new_a, new_b
            return

    # If there was no interval overlapping with the one we want to add, add
    # the new interval as a standalone interval to the list
    intervals.append((lower_bound, upper_bound))

while True:

    # Step 2.a: Starting the search
    if i == 1:
        s = Ceil(n, 3 * B)
        while True:

            c = (c_0 * pow(s, e, n)) % n
            if oracle.IsPaddingCorrect(c):
                break

            s += 1

    # Step 2.b: Searching with more than one interval left
    elif len(M) >= 2:
        while True:
            s += 1
            c = (c_0 * pow(s, e, n)) % n

            if oracle.IsPaddingCorrect(c):
                break

    # Step 2.c: Searching with one interval left
    elif len(M) == 1:
        a, b = M[0]

        # Check if the interval contains the solution
        if a == b:
            solution = a
            break

        r = Ceil(2 * (b * s - 2 * B), n)
        s = Ceil(2 * B + r * n, b)

        while True:
            c = (c_0 * pow(s, e, n)) % n
            if oracle.IsPaddingCorrect(c):
                break

            s += 1
            if s > (3 * B + r * n) // a:
                r += 1
                s = Ceil((2 * B + r * n), b)

    # Step 3: Narrowing the set of solutions
    M_new = []

    for a, b in M:
        min_r = Ceil(a * s - 3 * B + 1, n)
        max_r = (b * s - 2 * B) // n

        for r in range(min_r, max_r + 1):
            l = max(a, Ceil(2 * B + r * n, s))
            u = min(b, (3 * B - 1 + r * n) // s)

            if l > u:
                raise Exception('Unexpected error: l > u in step 3')

            # Do append and merge
            CalculateNewM(M_new, l, u)

    if len(M_new) == 0:
        raise Exception('Unexpected error: there are 0 intervals.')

    M = M_new
    i += 1

In [8]:
solution

5533002619174961876235287456749133721444485216543030105414479793856690210868449039941672630060633163429387649713088606627000843862898901848796639560960055660770496117244977814926921765800864397287174665246073288380004459330898744611622875155016473129657375289828709360796094721211345495472638126250615086

In [11]:
# Make sure the cracked message matches the original

cracked_message = solution.to_bytes(byteorder='big', length = BIT_STRENGTH//8)
print('-'*10, 'Cracked Message', '-'*10, '\n', cracked_message, '\n')
print('-'*10, 'Original Message', '-'*10, '\n', message, '\n')

---------- Cracked Message ---------- 
 b'\x00\x02\x04`\x00Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.' 

---------- Original Message ---------- 
 b'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.' 

